<a href="https://colab.research.google.com/github/markwreinke/OrganStopClassifier/blob/main/OrganStopClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the actual organ stop classifier program.

In [1]:
# Start with the imports
import pandas as pd
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook as tqdm
import os

In [2]:
# Mount GoogleDrive, cd and confirm current directory
from google.colab import drive
drive.mount("/content/drive")

dir = "/content/drive/My Drive/OrganStopDataset"
print(os.getcwd())

if(os.getcwd() != dir):
  os.chdir(dir)

print(os.getcwd())

Mounted at /content/drive
/content
/content/drive/My Drive/OrganStopDataset


In [4]:
# Here we are reading a comma separated values (CSV) file using pandas into the 
# variable df. df is type a 2D data structure with labeled axes, 
# as a DataFrame or TextParser
df = pd.read_csv('organstopClassifierDirectory.csv')

In [5]:
# Return the first n rows of the DataFrame. n defaults to 5 with no args
df.head()

,filename,fold,category 1,category 2
0,Krummhorn8AT4051Left-Live10 AT4051 Left.wav,1,Krummhorn8,AT4051
1,Krummhorn8AT4051Left-Live10 AT4051 Left-1.wav,2,Krummhorn8,AT4051
2,Krummhorn8AT4051Left-Live10 AT4051 Left-2.wav,3,Krummhorn8,AT4051
3,Krummhorn8AT4051Left-Live10 AT4051 Left-3.wav,4,Krummhorn8,AT4051
4,Krummhorn8AT4051Left-Live10 AT4051 Left-4.wav,5,Krummhorn8,AT4051


In [6]:
# The data was sorted into 5 "Cross-validation" folds, in which here, the first
# four are used as training data, and the fifth is a validation set
# Here's the paper: http://karol.piczak.com/papers/Piczak2015-ESC-Dataset.pdf
train = df[df['fold']!=5]
valid = df[df['fold']==5]

In [7]:
# Load a filewith librosa, and display its sampling rate and length
wav, sr = librosa.load('SampleDataset/Krummhorn8AT4051Left-Live10 AT4051 Left.wav', sr=None)
print(f'Sampling rate of the audio is {sr} and length of the audio is {len(wav)/sr} seconds')

Sampling rate of the audio is 44100 and length of the audio is 1.0 seconds


In [8]:
# Here we are going to convert a Spectrogram into an image
def spec_to_image(spec, eps=1e-6):
  mean = spec.mean()
  std = spec.std()
  spec_norm = (spec - mean) / (std + eps)
  spec_min, spec_max = spec_norm.min(), spec_norm.max()
  spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
  spec_scaled = spec_scaled.astype(np.uint8)
  return spec_scaled

  # This spectrogram is normalized using z score normalization and 
  # scaled using min-max scaling so its values lie between 0 and 255

In [9]:
# This function give a mel spectrogram of the given file
#  sr=None -> Librosa should use the native sampling rate of 44.1KHz to load the audio data instead of the default of 22.05KHz
#  Next the first 5 seconds of given audio is extracted
#  2048 samples are chosen for each window (about 46ms)
#  A hop_length of 512 samples is chose, meaning the window is moved by skipping 512 samples to get the next time frame
#  The number of mel filters is 128, makes the height of the spectrogram image 128
#  fmin and fmax are the lowest and highest frequencies
def get_melspectrogram_db(file_path, sr=None, n_fft=2048, hop_length=512, n_mels=128, fmin=20, fmax=8300, top_db=80):
  wav, sr = librosa.load(file_path, sr=sr)
  if wav.shape[0]<5*sr:
     wav=np.pad(wav,int(np.ceil((5*sr-wav.shape[0])/2)),mode='reflect')
  else:
    wav=wav[:5*sr]

  spec=librosa.feature.melspectrogram(wav, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, fmin=fmin, fmax=fmax)

  #Librosa squares the magnitude of the spectrogram when constructing Mel Spectrogram, so we use power_to_db to convert power magnitude to decipels. top_db is used to threshold the output.
  spec_db=librosa.power_to_db(spec, top_db=top_db)
  return spec_db

In [10]:
# This is us loading data into pytorch by building dataloaders to
# preprocess and load data.

class OrganStopSamples(Dataset):
  def __init__(self, base, df, in_col, out_col):
    self.df = df
    self.data = []
    self.labels = []
    self.c2i = {}
    self.i2c = {}
    self.categories = sorted(df[out_col].unique())
    for i, category in enumerate(self.categories):
      self.c2i[category] = i
      self.i2c[i] = category
    for ind in tqdm(range(len(df))):
      row = df.iloc[ind]
      file_path = os.path.join(base, row[in_col])
      # print("base: ", base, "\nin_col: ", in_col, "\nrow[in_col]: ", row[in_col], "\nfile_path:", file_path)
      self.data.append(spec_to_image(get_melspectrogram_db(file_path))[np.newaxis,...])
      self.labels.append(self.c2i[row['category 1']])

  def __len__(self):
     return len(self.data)
  def __getitem__(self, idx):
    return self.data[idx], self.labels[idx]

train_data = OrganStopSamples('SampleDataset', train, 'filename', 'category 1')
valid_data = OrganStopSamples('SampleDataset', valid, 'filename', 'category 1')
train_loader = DataLoader(train_data, batch_size = 16, shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = 16, shuffle = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


In [12]:
# This is its own model, not the resnet34
class OrganStopModel(nn.Module):
  def __init__(self, input_shape, batch_size=16, num_cats=50):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size = 3, stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(32)
    self.conv2 = nn.Conv2d(32, 32, kernel_size = 3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(32)
    self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
    self.bn3 = nn.BatchNorm2d(64)
    self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
    self.bn4 = nn.BatchNorm2d(64)
    self.conv5 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
    self.bn5 = nn.BatchNorm2d(128)
    self.conv6 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
    self.bn6 = nn.BatchNorm2d(128)
    self.conv7 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
    self.bn7 = nn.BatchNorm2d(256)
    self.conv8 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
    self.bn8 = nn.BatchNorm2d(256)
    self.dense1 = nn.Linear(256*(((input_shape[1]//2)//2)//2)*(((input_shape[2]//2)//2)//2),500)
    self.dropout = nn.Dropout(0.5)
    self.dense2 = nn.Linear(500, num_cats)
  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(self.bn1(x))
    x = self.conv2(x)
    x = F.relu(self.bn2(x))
    x = F.max_pool2d(x, kernel_size=2) 
    x = self.conv3(x)
    x = F.relu(self.bn3(x))
    x = self.conv4(x)
    x = F.relu(self.bn4(x))
    x = F.max_pool2d(x, kernel_size=2)
    x = self.conv5(x)
    x = F.relu(self.bn5(x))
    x = self.conv6(x)
    x = F.relu(self.bn6(x))
    x = F.max_pool2d(x, kernel_size=2)
    x = self.conv7(x)
    x = F.relu(self.bn7(x))
    x = self.conv8(x)
    x = F.relu(self.bn8(x))
    x = x.view(x.size(0),-1)
    x = F.relu(self.dense1(x))
    x = self.dropout(x)
    x = self.dense2(x)
    return x

In [13]:
if torch.cuda.is_available():
  device=torch.device('cuda:0')
else:
  device=torch.device('cpu')

In [14]:
model = OrganStopModel(input_shape=(1,128,431), batch_size=16, num_cats=50).to(device)

In [15]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 2e-5
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
epochs = 60
train_losses = []
valid_losses = []

In [16]:
def setlr(optimizer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr'] = lr

  return optimizer

In [18]:
def train(model, loss_fn, train_loader, valid_loader, epochs, optimizer, train_losses, valid_losses, change_lr=None):
  for epoch in tqdm(range(1,epochs+1)):
    model.train()
    batch_losses=[]
    if change_lr:
      optimizer = change_lr(optimizer, epoch)
    for i, data in enumerate(train_loader):
      x, y = data
      optimizer.zero_grad()
      x = x.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long)
      y_hat = model(x)
      loss = loss_fn(y_hat, y)
      loss.backward()
      batch_losses.append(loss.item())
      optimizer.step()
    train_losses.append(batch_losses)
    print(f'Epoch - {epoch} Train-Loss : {np.mean(train_losses[-1])}')
    model.eval()
    batch_losses=[]
    trace_y = []
    trace_yhat = []
    for i, data in enumerate(valid_loader):
      x, y = data
      x = x.to(device, dtype=torch.float32)
      y = y.to(device, dtype=torch.long)
      y_hat = model(x)
      loss = loss_fn(y_hat, y)
      trace_y.append(y.cpu().detach().numpy())
      trace_yhat.append(y_hat.cpu().detach().numpy())      
      batch_losses.append(loss.item())
    valid_losses.append(batch_losses)
    trace_y = np.concatenate(trace_y)
    trace_yhat = np.concatenate(trace_yhat)
    accuracy = np.mean(trace_yhat.argmax(axis=1)==trace_y)
    print(f'Epoch - {epoch} Valid-Loss : {np.mean(valid_losses[-1])} Valid-Accuracy : {accuracy}')

In [19]:
def lr_decay(optimizer, epoch):
  if epoch%20==0:
    new_lr = learning_rate / (10**(epoch//20))
    optimizer = setlr(optimizer, new_lr)
    print(f'Changed learning rate to {new_lr}')
  return optimizer
train(model, loss_fn, train_loader, valid_loader, epochs, optimizer, train_losses, valid_losses, lr_decay)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Epoch - 1 Train-Loss : 0.08836698566976083
Epoch - 1 Valid-Loss : 0.04857312029258463 Valid-Accuracy : 0.9968652037617555
Epoch - 2 Train-Loss : 0.01248768050005199
Epoch - 2 Valid-Loss : 0.10595577701914338 Valid-Accuracy : 0.9968652037617555
Epoch - 3 Train-Loss : 0.0038507962729612656
Epoch - 3 Valid-Loss : 0.05476015793601503 Valid-Accuracy : 0.9968652037617555
Epoch - 4 Train-Loss : 0.00025641708582067044
Epoch - 4 Valid-Loss : 0.04681281300875686 Valid-Accuracy : 0.9968652037617555
Epoch - 5 Train-Loss : 0.0038475689352148678
Epoch - 5 Valid-Loss : 0.12244336835979926 Valid-Accuracy : 0.9968652037617555
Epoch - 6 Train-Loss : 0.0002998857708877978
Epoch - 6 Valid-Loss : 0.07026953741892425 Valid-Accuracy : 0.9968652037617555
Epoch - 7 Train-Loss : 3.4525037136756695e-05
Epoch - 7 Valid-Loss : 0.07963233776390495 Valid-Accuracy : 0.9968652037617555
Epoch - 8 Train-Loss : 0.0008871149815874557
Epoch - 8 Valid-Loss : 0.12789289361247072 Valid-Accuracy : 0.9968652037617555
Epoch - 9 